In [1]:
import glob
import tensorflow

print(glob)
print(tensorflow.__version__)

<module 'glob' from '/opt/conda/lib/python3.9/glob.py'>
2.6.0


### 데이터 읽어오기

In [2]:
import os, re 
import numpy as np
import tensorflow as tf

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)
    
raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ["Busted flat in Baton Rouge, waitin' for a train", "And I's feelin' near as faded as my jeans", 'Bobby thumbed a diesel down, just before it rained']


### 데이터 정제

앞서 배운 테크닉들을 활용해 문장 생성에 적합한 모양새로 데이터를 정제하세요!

preprocess_sentence() 함수를 만든 것을 기억하시죠? 이를 활용해 데이터를 정제하도록 하겠습니다.

추가로 지나치게 긴 문장은 다른 데이터들이 과도한 Padding을 갖게 하므로 제거합니다. 너무 긴 문장은 노래 가사 작사하기에 어울리지 않을 수도 있겠죠.
그래서 이번에는 문장을 토큰화 했을 때 토큰의 개수가 15개를 넘어가는 문장을 학습 데이터에서 제외하기 를 권합니다.

In [3]:
for idx, sentence in enumerate(raw_corpus): #enumerate 열거하다
    if len(sentence.split(" ")) >= 15 : continue  #길이가 15인 문장은 건너뜁니다.
    if len(sentence) == 0: continue   # 길이가 0인 문장은 건너뜁니다.
    if sentence[-1] == ":": continue  # 문장의 끝이 : 인 문장은 건너뜁니다.    
    if idx > 9: break   # 일단 문장 10개만 확인해 볼 겁니다.
        
    print(sentence)

Busted flat in Baton Rouge, waitin' for a train
And I's feelin' near as faded as my jeans
Bobby thumbed a diesel down, just before it rained
I was playin' soft while Bobby sang the blues, yeah
Windshield wipers slappin' time, I was holdin' Bobby's hand in mine
Nothin', don't mean nothin' hon' if it ain't free, no no
And, feelin' good was easy, Lord, when he sang the blues
You know, feelin' good was good enough for me


이런 전처리를 위해 정규표현식(Regex)을 이용한 필터링이 유용하게 사용됩니다.

In [4]:
# 입력된 문장을
#     1. 소문자로 바꾸고, 양쪽 공백을 지웁니다
#     2. 특수문자 양쪽에 공백을 넣고
#     3. 여러개의 공백은 하나의 공백으로 바꿉니다
#     4. a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꿉니다
#     5. 다시 양쪽 공백을 지웁니다
#     6. 문장 시작에는 <start>, 끝에는 <end>를 추가합니다
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence

# 이 문장이 어떻게 필터링되는지 확인해 보세요.
print(preprocess_sentence("This @_is ;;;sample        sentence."))

<start> this is sample sentence . <end>


In [5]:
corpus = []

for sentence in raw_corpus:
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
    preprocessed_sentence = preprocess_sentence(sentence)
    if len(sentence.split(" ")) >= 15 : continue #길이가 15이상인 것은 건너뛴다.
    corpus.append(preprocessed_sentence)
   
corpus[:10]

['<start> busted flat in baton rouge , waitin for a train <end>',
 '<start> and i s feelin near as faded as my jeans <end>',
 '<start> bobby thumbed a diesel down , just before it rained <end>',
 '<start> i was playin soft while bobby sang the blues , yeah <end>',
 '<start> windshield wipers slappin time , i was holdin bobby s hand in mine <end>',
 '<start> nothin , don t mean nothin hon if it ain t free , no no <end>',
 '<start> and , feelin good was easy , lord , when he sang the blues <end>',
 '<start> you know , feelin good was good enough for me <end>',
 '<start> there bobby shared the secrets of my soul <end>',
 '<start> through all kinds of weather , through everything we done <end>']

In [6]:
def tokenize(corpus):
    # 12000단어를 기억할 수 있는 tokenizer를 만들겁니다
    # 12000단어에 포함되지 못한 단어는 '<unk>'로 바꾼다.
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000, 
        filters=' ',
        oov_token="<unk>"
    )
    # corpus를 이용해 tokenizer 내부의 단어장을 완성합니다
    tokenizer.fit_on_texts(corpus)
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환합니다
    tensor = tokenizer.texts_to_sequences(corpus)   
    # 입력 데이터의 시퀀스 길이를 일정하게 맞춰줍니다
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post',   maxlen=15)  #토큰수를 15개 이하로 출력
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2 4067 1727 ...    0    0    0]
 [   2    8    5 ...    0    0    0]
 [   2  883 7177 ...    0    0    0]
 ...
 [   5   22  716 ... 3882    4    3]
 [   5   22  716 ... 3882    4    3]
 [   5   22  716 ... 3882   19    3]] <keras_preprocessing.text.Tokenizer object at 0x7f91ea60c730>


In [7]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : i
6 : the
7 : you
8 : and
9 : a
10 : to


토큰으로 잘 쪼개졌다. 

In [8]:
print(tensor[:3, :16])
# 생성된 텐서 데이터를 3번째 행, 16번째 열까지만 출력해 봅시다. 
#토큰의 갯수가 16개까지 출력하기로 했지만 maxlen=15으로 인해 15개만 출력이 된다. 

[[    2  4067  1727    14 10766  4505     4  1181    28     9   658     3
      0     0     0]
 [    2     8     5    16   512   878    81  2544    81    13   892     3
      0     0     0]
 [    2   883  7177     9  5647    59     4    35   178    11  4263     3
      0     0     0]]


In [9]:
src_input = tensor[:, :-1]  # tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다. 마지막 토큰은 <END>가 아니라 <pad>일 가능성이 높습니다.
tgt_input = tensor[:, 1:]    # tensor에서 <START>를 잘라내서 타겟 문장을 생성합니다.

print(src_input[0]) #첫번째 행 맨뒤 꺼 하나 '0'뺀거
print(tgt_input[0]) #첫번째 행, 맨앞 꺼 하나 '2'뺀거

[    2  4067  1727    14 10766  4505     4  1181    28     9   658     3
     0     0]
[ 4067  1727    14 10766  4505     4  1181    28     9   658     3     0
     0     0]


## 평가 데이터셋 분리

데이터가 텐서로 변환되었다. 

 sklearn 모듈의 train_test_split() 함수를 사용해 훈련 데이터와 평가 데이터를 분리하도록 하겠습니다. 단어장의 크기는 12,000 이상 으로 설정하세요! 총 데이터의 20% 를 평가 데이터셋으로 사용해 주세요!

In [10]:
from sklearn.model_selection import train_test_split
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, 
                                                          tgt_input,
                                                          test_size=0.2,
                                                          shuffle=True, 
                                                          random_state=1)

In [11]:
print('encoding Train: ', enc_train.shape)
print('decoding Train: ', dec_train.shape)

encoding Train:  (132788, 14)
decoding Train:  (132788, 14)


In [12]:
BUFFER_SIZE = len(enc_train)
BATCH_SIZE = 256
steps_per_epoch = len(enc_train) // BATCH_SIZE

In [13]:
dataset = tf.data.Dataset.from_tensor_slices((enc_train, dec_train))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

### 인공지능 만들기

 모델의 Embedding Size와 Hidden Size를 조절하며 10 Epoch 안에 val_loss 값을 2.2 수준으로 줄일 수 있는 모델을 설계하세요! (Loss는 아래 제시된 Loss 함수를 그대로 사용!)

그리고 멋진 모델이 생성한 가사 한 줄을 제출하시길 바랍니다!

 tf.keras.Model을 Subclassing 한다.

In [14]:
from tensorflow import keras
from tensorflow.keras import layers

In [15]:
from tensorflow.keras.layers import Embedding, LSTM, Dense

In [16]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super(TextGenerator, self).__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.batnorm = tf.keras.layers.BatchNormalization() #배치 정규화
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
    def call(self, x):
        out = self.embedding(x)
        out = self.batnorm(out)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 600
hidden_size = 2400
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

### 모델 학습

In [21]:
history = []
epochs = 10

optimizer = tf.keras.optimizers.Adam()

loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)
model.compile(loss=loss, optimizer=optimizer)

history = model.fit(enc_train, 
          dec_train, 
          epochs=epochs,
          batch_size=256,
          validation_data=(enc_val, dec_val),
          verbose=1)

Epoch 1/10
519/519 [==============================] - 374s 673ms/step - loss: 3.1961 - val_loss: 4.8063
Epoch 2/10
519/519 [==============================] - 349s 672ms/step - loss: 2.6836 - val_loss: 2.6348
Epoch 3/10
519/519 [==============================] - 349s 673ms/step - loss: 2.3531 - val_loss: 2.4591
Epoch 4/10
519/519 [==============================] - 349s 673ms/step - loss: 2.0420 - val_loss: 2.3399
Epoch 5/10
519/519 [==============================] - 350s 674ms/step - loss: 1.7545 - val_loss: 2.2600
Epoch 6/10
519/519 [==============================] - 350s 674ms/step - loss: 1.5017 - val_loss: 2.2130
Epoch 7/10
519/519 [==============================] - 350s 675ms/step - loss: 1.2937 - val_loss: 2.1948
Epoch 8/10
519/519 [==============================] - 350s 674ms/step - loss: 1.1360 - val_loss: 2.2046
Epoch 9/10
519/519 [==============================] - 350s 674ms/step - loss: 1.0332 - val_loss: 2.2234
Epoch 10/10
519/519 [==============================] - 350s 674m

In [17]:
history = []
epochs = 7

optimizer = tf.keras.optimizers.Adam()

loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)
model.compile(loss=loss, optimizer=optimizer)

history = model.fit(enc_train, 
          dec_train, 
          epochs=epochs,
          batch_size=256,
          validation_data=(enc_val, dec_val),
          verbose=1)

Epoch 1/7
519/519 [==============================] - 395s 709ms/step - loss: 3.1697 - val_loss: 3.3958
Epoch 2/7
519/519 [==============================] - 371s 716ms/step - loss: 2.6522 - val_loss: 2.6062
Epoch 3/7
519/519 [==============================] - 372s 717ms/step - loss: 2.3094 - val_loss: 2.4313
Epoch 4/7
519/519 [==============================] - 372s 716ms/step - loss: 1.9851 - val_loss: 2.3093
Epoch 5/7
519/519 [==============================] - 372s 717ms/step - loss: 1.6873 - val_loss: 2.2315
Epoch 6/7
519/519 [==============================] - 372s 717ms/step - loss: 1.4305 - val_loss: 2.1909
Epoch 7/7
519/519 [==============================] - 372s 717ms/step - loss: 1.2275 - val_loss: 2.1850


epoch 7 에서 val loss값이 2.2 이하로 떨어졌다.

In [18]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [19]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you so much , so , i do <end> '

나는 너를 매우 좋아해 그래서 나는 한다. 라는 멋진 문장이 나왔다. 

In [20]:
generate_text(model, tokenizer, init_sentence="<start> she", max_len=20)

'<start> she s got me runnin round and round <end> '

그녀는 나를 둥글게 둥글게 해 라는 문장이 나왔다.

In [21]:
generate_text(model, tokenizer, init_sentence="<start> jappari", max_len=20)

'<start> <unk> <unk> <unk> <unk> <end> '

jappari 라는 단어장에 없는 단어를 쓰면 \<unk>가 뜬다.

### 회고

val - loss 를 2.2 이하로 내리기가 매우 힘들었던것 같다. 처음에 긴 문장은 자르고 maxlen으로 단어를 더 쳐내다 보니 아무래도 참고 데이터 량이 줄어서 그런지 embedding_size,와 hidden size를 크게하면 과적합이 일어났다.<br> 하지만, 과적합을 방지하기 위해 
self.batnorm = tf.keras.layers.BatchNormalization() #배치 정규화를 써주니 그래도 과적합이 일어나는게 조금은 줄어들었고 2.2 이하로 내리는데 성공하였다. <br>
중간에 여러 오류들이 있었지만 해결해 나가는 과정도 꽤나 재밌었던것 같다.